# Draft Optimization

This notebook evaluates champion combinations using the local LoL draft prediction model to find the optimal team compositions.

In [ ]:
import pandas as pd
import requests
import itertools
from tqdm.notebook import tqdm
import os
from dotenv import load_dotenv

from utils.rl.champions import Champion

# Load API key once at the start
load_dotenv()
API_KEY = os.getenv("API_KEY")
HEADERS = {"X-API-Key": API_KEY} if API_KEY else {}

In [ ]:
patch = "15.04"
numerical_elo = 0  # highest numerical elo

## Define Champion Pools

Define the champion pools for each role. Starting with 3 champions per role.

In [ ]:
#TODO: Could also do combinations for ennemy champions, but understand that they should be limited: probably against key meta champions!

In [ ]:
# Los ratones champion pools
champion_pools = {
    "TOP": [
        Champion.JAX,
        Champion.SION,
        Champion.VOLIBEAR,
        Champion.GRAGAS,
        Champion.QUINN,
        Champion.CHOGATH,
        Champion.GAREN,
        Champion.POPPY,
        Champion.VI,
        Champion.AMBESSA,
    ],
    "JUNGLE": [
        Champion.JARVAN_IV,
        Champion.VIEGO,
        Champion.WUKONG,
        Champion.PANTHEON,
        Champion.VI,
        Champion.IVERN,
        Champion.MAOKAI,
    ],
    "MID": [
        Champion.GALIO,
        Champion.ORIANNA,
        Champion.AZIR,
        Champion.SYNDRA,
        Champion.AHRI,
        Champion.VIKTOR,
        Champion.HWEI,
        Champion.MEL,
        Champion.ZILEAN,
        Champion.CHOGATH,
    ],
    "BOT": [
        Champion.CORKI,
        Champion.EZREAL,
        Champion.JINX,
        Champion.KALISTA,
        Champion.TRISTANA,
        Champion.ZERI,
        Champion.SIVIR,
        Champion.XAYAH,
    ],
    "SUPPORT": [
        Champion.BRAUM,
        Champion.JANNA,
        Champion.RAKAN,
        Champion.RELL,
        Champion.ALISTAR,
        Champion.RENATA_GLASC,
        Champion.LULU,
        Champion.MILIO,
        Champion.BARD,
    ],
}

# Create lookup dictionaries
id_to_name = {champion.id: champion.display_name for champion in Champion}
name_to_id = {champion.display_name: champion.id for champion in Champion}

# Display the champion pools
for role, champions in champion_pools.items():
    print(f"{role}: {', '.join([champion.display_name for champion in champions])}")

## Generate Team Compositions

Generate all possible team compositions from the champion pools.

In [ ]:
def generate_team_comps(champion_pools):
    """
    Generate all possible team compositions from the champion pools,
    excluding compositions where the same champion appears in multiple roles.

    Args:
        champion_pools: Dictionary of champion pools for each role

    Returns:
        List of team compositions, where each composition is a list of 5 unique champion objects
    """
    roles = ["TOP", "JUNGLE", "MID", "BOT", "SUPPORT"]
    # Generate all possible combinations
    all_comps = list(itertools.product(*[champion_pools[role] for role in roles]))

    # Filter out compositions with duplicate champions
    valid_comps = []
    for comp in all_comps:
        # Create a set of champion IDs to check for uniqueness
        champion_ids = {champion.id for champion in comp}
        # If we have 5 unique champions, this is a valid composition
        if len(champion_ids) == 5:
            valid_comps.append(comp)

    return valid_comps


# Generate all possible team compositions
team_comps = generate_team_comps(champion_pools)
print(f"Generated {len(team_comps)} possible team compositions")

## Setup Model Prediction Functions

Create functions to get predictions from the model API.

In [ ]:
def get_prediction(champion_ids, side="blue"):
    """
    Get winrate prediction for a team composition.

    Args:
        champion_ids: List of 5 champion IDs for the team
        side: 'blue' or 'red'

    Returns:
        Winrate prediction (0-1)
    """
    # Create the full 10-champion array with UNKNOWNs for the opponent team
    full_champion_ids = []

    if side == "blue":
        # Team on blue side (positions 0-4)
        full_champion_ids = champion_ids + ["UNKNOWN"] * 5
    else:
        # Team on red side (positions 5-9)
        full_champion_ids = ["UNKNOWN"] * 5 + champion_ids

    # Prepare the API request
    api_input = {
        "champion_ids": full_champion_ids,
        "numerical_elo": numerical_elo,
        "patch": patch,
    }

    # Make the API request
    try:
        response = requests.post(
            "http://0.0.0.0:8000/predict", json=api_input, headers=HEADERS
        )
        response.raise_for_status()
        if side == "blue":
            return response.json()["win_probability"]
        elif side == "red":
            return 1 - response.json()["win_probability"]
        else:
            raise ValueError("invalide side")
    except requests.exceptions.RequestException as e:
        print(f"Error making prediction: {e}")
        return None

## Evaluate Team Compositions

Evaluate all team compositions by checking their winrates on both blue and red sides.

In [ ]:
# Evaluate all team compositions
def evaluate_team_comps():
    results = []

    # Use tqdm for a progress bar
    for comp in tqdm(team_comps, desc="Evaluating team compositions"):
        # Extract champion IDs
        champion_ids = [champion.id for champion in comp]
        champion_names = [champion.display_name for champion in comp]

        # Get predictions for both sides
        blue_winrate = get_prediction(champion_ids, side="blue")
        red_winrate = get_prediction(champion_ids, side="red")

        if blue_winrate is not None and red_winrate is not None:
            # Calculate average winrate
            avg_winrate = (blue_winrate + red_winrate) / 2

            # Create a result entry
            result = {
                "TOP": champion_names[0],
                "JUNGLE": champion_names[1],
                "MID": champion_names[2],
                "BOT": champion_names[3],
                "SUPPORT": champion_names[4],
                "blue_winrate": blue_winrate,
                "red_winrate": red_winrate,
                "avg_winrate": avg_winrate,
            }

            results.append(result)

    return results


# Run the evaluation
# results = evaluate_team_comps()

## Optimized Batch Processing

Here's an optimized version that uses batched API calls for better performance.

In [ ]:
# Function for batched processing (uncomment to use after confirming the simple version works)
def evaluate_team_comps_batched():
    # Create a list to hold all API requests we need to make
    all_requests = []

    # For each composition, we need to evaluate it on both blue and red side
    for idx, comp in enumerate(team_comps):
        champion_ids = [champion.id for champion in comp]

        # Request for blue side (comp is on blue side)
        blue_request = {
            "champion_ids": champion_ids + ["UNKNOWN"] * 5,
            "numerical_elo": numerical_elo,
            "patch": patch,
        }

        # Request for red side (comp is on red side)
        red_request = {
            "champion_ids": ["UNKNOWN"] * 5 + champion_ids,
            "numerical_elo": numerical_elo,
            "patch": patch,
        }

        # Add to our request list with metadata
        all_requests.append((blue_request, "blue", idx))
        all_requests.append((red_request, "red", idx))

    # Organize results
    blue_results = {}
    red_results = {}

    # Process in batches
    BATCH_SIZE = 16
    with tqdm(total=len(all_requests), desc="Making batch API calls") as pbar:
        for i in range(0, len(all_requests), BATCH_SIZE):
            batch = all_requests[i : i + BATCH_SIZE]

            # Extract just the API requests
            api_requests = [req[0] for req in batch]

            # Make the batch API call
            try:
                response = requests.post(
                    "http://0.0.0.0:8000/predict-batch",
                    json=api_requests,
                    headers=HEADERS,
                )
                response.raise_for_status()

                # Process results
                for (_, side, idx), prediction in zip(batch, response.json()):
                    winrate = prediction["win_probability"]

                    if side == "blue":
                        blue_results[idx] = winrate
                    else:
                        red_results[idx] = 1 - winrate

            except Exception as e:
                print(f"Error in batch processing: {e}")

            pbar.update(len(batch))

    # Combine results
    results = []
    for idx, comp in enumerate(team_comps):
        if idx in blue_results and idx in red_results:
            champion_names = [champion.display_name for champion in comp]
            champion_ids_list = [champion.id for champion in comp]
            blue_winrate = blue_results[idx]
            red_winrate = red_results[idx]
            avg_winrate = (blue_winrate + red_winrate) / 2

            results.append(
                {
                    "TOP": champion_names[0],
                    "JUNGLE": champion_names[1],
                    "MID": champion_names[2],
                    "BOT": champion_names[3],
                    "SUPPORT": champion_names[4],
                    "TOP_ID": champion_ids_list[0],
                    "JUNGLE_ID": champion_ids_list[1],
                    "MID_ID": champion_ids_list[2],
                    "BOT_ID": champion_ids_list[3],
                    "SUPPORT_ID": champion_ids_list[4],
                    "blue_winrate": blue_winrate,
                    "red_winrate": red_winrate,
                    "avg_winrate": avg_winrate,
                }
            )

    return results


# Uncomment to use the batched version instead
results = evaluate_team_comps_batched()

## Analyze Results

Create a DataFrame with the results and display the top team compositions.

In [ ]:
# Create DataFrame from results
df = pd.DataFrame(results)

# Sort by average winrate
df = df.sort_values(by='avg_winrate', ascending=False)

# Display top 10 compositions
print("Top 10 Team Compositions:")
display(df.head(10))

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_enhanced_browser(df):
    # Create filter widgets for each role
    include_filters = {}
    exclude_filters = {}
    
    for role in ['TOP', 'JUNGLE', 'MID', 'BOT', 'SUPPORT']:
        # Get unique champions for this role
        unique_champs = sorted(df[role].unique())
        
        # Create include filter (without description to fix alignment)
        include_filters[role] = widgets.SelectMultiple(
            options=unique_champs,
            layout=widgets.Layout(width='180px', height='120px')
        )
        
        # Create exclude filter (without description to fix alignment)
        exclude_filters[role] = widgets.SelectMultiple(
            options=unique_champs,
            layout=widgets.Layout(width='180px', height='120px')
        )
    
    # Create pagination controls
    page_size = widgets.BoundedIntText(
        value=10,
        min=1,
        max=100,
        description='Page size:',
        layout=widgets.Layout(width='150px')
    )
    
    current_page = widgets.BoundedIntText(
        value=1,
        min=1,
        description='Page:',
        layout=widgets.Layout(width='150px')
    )
    
    prev_button = widgets.Button(
        description='Previous',
        layout=widgets.Layout(width='100px')
    )
    
    next_button = widgets.Button(
        description='Next',
        layout=widgets.Layout(width='100px')
    )
    
    # Create output area for displaying results
    output = widgets.Output()
    
    # Clear filter buttons for each role
    clear_buttons = {}
    for role in ['TOP', 'JUNGLE', 'MID', 'BOT', 'SUPPORT']:
        clear_buttons[role] = widgets.Button(
            description='Clear Filters',
            layout=widgets.Layout(width='180px')
        )
    
    # Function to apply filters and update display
    def update_display():
        with output:
            clear_output(wait=True)
            
            # Apply filters for each role
            filtered_df = df.copy()
            
            for role in ['TOP', 'JUNGLE', 'MID', 'BOT', 'SUPPORT']:
                include_champs = list(include_filters[role].value)
                exclude_champs = list(exclude_filters[role].value)
                
                # Apply include filter (if any champions selected)
                if include_champs:
                    filtered_df = filtered_df[filtered_df[role].isin(include_champs)]
                
                # Apply exclude filter (if any champions selected)
                if exclude_champs:
                    filtered_df = filtered_df[~filtered_df[role].isin(exclude_champs)]
            
            # Calculate total pages
            total_rows = len(filtered_df)
            total_pages = max(1, (total_rows + page_size.value - 1) // page_size.value)
            
            # Update current page if needed
            if current_page.value > total_pages:
                current_page.value = total_pages
            
            # Calculate slice for current page
            start_idx = (current_page.value - 1) * page_size.value
            end_idx = min(start_idx + page_size.value, total_rows)
            
            # Display pagination info
            print(f"Showing {start_idx+1}-{end_idx} of {total_rows} results (Page {current_page.value} of {total_pages})")
            
            # Display the filtered and paginated data
            if total_rows > 0:
                display(filtered_df.iloc[start_idx:end_idx])
            else:
                print("No results match your filters.")
    
    # Connect button events
    def on_prev_click(b):
        if current_page.value > 1:
            current_page.value -= 1
            update_display()
    
    def on_next_click(b):
        # The max value will be calculated in update_display
        current_page.value += 1
        update_display()
    
    # Create clear filter handlers
    def create_clear_handler(role):
        def clear_handler(b):
            include_filters[role].value = ()
            exclude_filters[role].value = ()
            update_display()
        return clear_handler
    
    # Connect events
    prev_button.on_click(on_prev_click)
    next_button.on_click(on_next_click)
    
    # Connect clear buttons
    for role in clear_buttons:
        clear_buttons[role].on_click(create_clear_handler(role))
    
    # Update when any filter or pagination control changes
    for role in include_filters:
        include_filters[role].observe(lambda change: update_display(), names='value')
        exclude_filters[role].observe(lambda change: update_display(), names='value')
    
    page_size.observe(lambda change: update_display(), names='value')
    current_page.observe(lambda change: update_display(), names='value')
    
    # Create layout
    filter_boxes = []
    for role in ['TOP', 'JUNGLE', 'MID', 'BOT', 'SUPPORT']:
        # Create proper headers with HTML
        include_header = widgets.HTML(value=f"<b>Include {role}:</b>")
        exclude_header = widgets.HTML(value=f"<b>Exclude {role}:</b>")
        
        # Layout each role's filters in a vertical box
        filter_box = widgets.VBox([
            widgets.HTML(value=f"<h4>{role}</h4>"),
            include_header,
            include_filters[role],
            exclude_header,
            exclude_filters[role],
            clear_buttons[role]
        ], layout=widgets.Layout(margin='0px', padding='0px'))
        
        filter_boxes.append(filter_box)
    
    filters_row = widgets.HBox(filter_boxes, layout=widgets.Layout(margin='0px', padding='0px'))
    
    pagination_row = widgets.HBox([
        page_size,
        current_page,
        prev_button,
        next_button
    ])
    
    # Main layout
    main_layout = widgets.VBox([
        widgets.HTML(value="<h3>Team Composition Browser</h3>"),
        filters_row,
        pagination_row,
        output
    ])
    
    # Initial display
    display(main_layout)
    update_display()

# Use the function with your results DataFrame
create_enhanced_browser(df)

In [ ]:
df.info()

# Export to sqlite

In [ ]:
# Export results to SQLite with champion IDs
import sqlite3

# Create SQLite database
db_path = "lol_team_compositions.db"
conn = sqlite3.connect(db_path)

# Create table with proper indices including champion IDs
conn.execute(
    """
CREATE TABLE team_comps (
    id INTEGER PRIMARY KEY,
    TOP TEXT NOT NULL,
    JUNGLE TEXT NOT NULL, 
    MID TEXT NOT NULL,
    BOT TEXT NOT NULL,
    SUPPORT TEXT NOT NULL,
    TOP_ID TEXT NOT NULL,
    JUNGLE_ID TEXT NOT NULL,
    MID_ID TEXT NOT NULL,
    BOT_ID TEXT NOT NULL,
    SUPPORT_ID TEXT NOT NULL,
    blue_winrate REAL NOT NULL,
    red_winrate REAL NOT NULL,
    avg_winrate REAL NOT NULL
)
"""
)

# Create indices for faster filtering
conn.execute("CREATE INDEX idx_top ON team_comps(TOP)")
conn.execute("CREATE INDEX idx_jungle ON team_comps(JUNGLE)")
conn.execute("CREATE INDEX idx_mid ON team_comps(MID)")
conn.execute("CREATE INDEX idx_bot ON team_comps(BOT)")
conn.execute("CREATE INDEX idx_support ON team_comps(SUPPORT)")
conn.execute("CREATE INDEX idx_top_id ON team_comps(TOP_ID)")
conn.execute("CREATE INDEX idx_jungle_id ON team_comps(JUNGLE_ID)")
conn.execute("CREATE INDEX idx_mid_id ON team_comps(MID_ID)")
conn.execute("CREATE INDEX idx_bot_id ON team_comps(BOT_ID)")
conn.execute("CREATE INDEX idx_support_id ON team_comps(SUPPORT_ID)")
conn.execute("CREATE INDEX idx_avg_winrate ON team_comps(avg_winrate)")

# Insert data
df.to_sql("team_comps", conn, if_exists="append", index=False)

# Commit and close
conn.commit()
conn.close()

print(f"SQLite database created at: {db_path}")